## Neural Network

In [ ]:
# Let's start by importing the relevant packages
# matplotlib for plots
import matplotlib as mpl
from matplotlib import pyplot as plt
# pandas to read in some data
import pandas as pd
# numpy to build our first perceptron
import numpy as np
# Train test split to do validate our findings from the perceptron training
from sklearn.model_selection import train_test_split
# MinMaxScaler to normalise the data before inputting them to the perceptron
from sklearn.preprocessing import MinMaxScaler
# PyTorch for neural networks
import torch
import time
from torch import nn
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 9)
import os
home = os.path.expanduser("~")
data = home + '/data/workshop_data/occupancy_data/datatraining.txt'


In [ ]:
# Load the occupancy data so we have something to predict
df = pd.read_csv(data)
target = 'Occupancy'
features = [col for col in df.columns if target not in col and 'date' not in col]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(df[features], df[target], shuffle=False)
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

## Build the neural network
To extend our previously build neuron to a neural network, we will need to add a second (third, fourth) linear layer.

The first layer needs to output as many layers as the second one consumes. Try 10 for the time being.
You will need to update the logits and forward function as well to pass through all layers.


In [ ]:
class Network(nn.Module):
    
    def __init__(self, number_of_inputs, hidden_units):
        super().__init__()
        # Build the network using nn.Linear
        self.linear1 = 
        self.linear2 = 
        # use nn.Sigmoid as an activation function
        self.act1 = 
        self.act2 = 
    
    def logit(self, inp):
        # Calculate the logits as the input to the last activation
    
    def forward(self, inp):
        # Calculate the output of the whole network utilising the logit function
        
    

Let us now select a random selection of the training data and calculate the gradients for the neuron:

In [ ]:
loss = nn.BCEWithLogitsLoss()
net = Network(5, 10)
optim = torch.optim.SGD(net.parameters(), lr=5e-2)

In [ ]:
def fit_batch(optim, loss, net, x, y):
    net.train()
    optim.zero_grad()
    y_pred = net.logit(x)
    err = loss(y_pred, y)
    err.mean().backward()
    optim.step()
    return y_pred

def eval_batch(net, x):
    net.eval()
    y_pred = net(x)
    return y_pred

In [ ]:
if torch.cuda.is_available():
    net = Network(5, 10).cuda()
    optim = torch.optim.Adam(net.parameters(), lr=1e-2)
    start = time.time()
    for i in range(20):
        acc = None
        for i in range(200):
            select = np.random.randint(0, len(x_train), 2048)
            x = torch.from_numpy(x_train[select]).float().cuda()
            y = torch.from_numpy(y_train.iloc[select].values).float().unsqueeze(1).cuda()
            y_pred = fit_batch(optim, loss, net, x, y)
            if acc is None:
                acc = (y==(y_pred > .5).float()).float().mean()
            else:
                acc += (y==(y_pred > .5).float()).float().mean()
        print('train accuracy {}'.format(acc.data.cpu().numpy()/200))
    
    x = torch.from_numpy(x_val).float().cuda()
    y = torch.from_numpy(y_val.values).float().unsqueeze(1).cuda()
    y_pred = eval_batch(net, x)
    acc = (y==(y_pred > .5).float()).float().mean()
    print('val accuracy {}'.format(acc.data.cpu().numpy()))
    
    print('Training time: {}'.format(time.time() - start))

## How does the result change with a changing network?

Now try using a bigger layersize and try adding dropout.
How can we change the training and validation loss?

- What happens if we add Dropout? [docs](https://pytorch.org/docs/stable/nn.html#dropout)
- What happens if you add momentum or weight decay to SGD? [docs](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD)
- What happens if you use an Adam optimizer instead of SGD? [docs](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam)
- What happens if we use other activation functions? [docs](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)

Hint You can add dropout using:
```
    self.dropout = nn.Dropout(how_many_percent_shall_be_dropped)
    
    def logits(x):
        out_layer1 = self.dropout(self.act1(self.linear1(x)))
        ...
```